In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import time
import datetime
import ta
def upper_shadow(opener, close, high):
    if opener >= close:
        return abs(opener-high)
    else:
        return abs(close-high)

def lower_shadow(opener, close, low):
    if opener <= close:
        return abs(opener-low)
    else:
        return abs(close-low)

def real_body(opener, close):
    return abs(opener-close)

def total_dollar(av,bv,a,b):
    p = (a+b)/2.0
    v = av+bv
    return p*v

def log_dollar(a,b):
    p = (a+b)/2.0
    return np.log(p)

def volumen(av,bv):
    return av+bv

def price(a,b):
    g = (a+b)/2.0
    return b

#df = pd.read_csv('Dec.csv')
#df["Dollar"] = df.apply(lambda x: total_dollar(x.AskVolume,x.BidVolume,x.Ask,x.Bid), axis = 1)
def tick_preprocess(df):
    df["Price"] = df.apply(lambda x: price(x.Ask,x.Bid), axis = 1)
    return df
#df.to_csv("test_v6.csv",index=False)

In [2]:
def indicator_fill(df):
    a = ta.trend.ema_indicator(close=df.Close,window=5)
    f = ta.trend.sma_indicator(close=df.Close,window=50)
    i = ta.trend.sma_indicator(close=df.Close,window=20)
    c = ta.volatility.average_true_range(high=df.High,low=df.Low,close=df.Close)
    df["5_EMA"] = a
    df["50_SMA"] = f
    df["20_SMA"] = i
    df['ATR'] = c
    return df


In [3]:
def buy_calc_side_v(hi,lo,clo,start):
    if statistics.median(clo[1:]) > start:
        return "buy"
    else:
        return "wrong buy"

def sell_calc_side_v(hi,lo,clo,start):
    if statistics.median(clo[1:]) < start:
        return "sell"
    else:
        return "wrong sell"

In [4]:
import math
import statistics
def buy_quality(hi,lo,clo,start,bl):
    if bl =='buy' and abs(max(hi[1:]) - start) > abs(min(lo[1:]) - start):
        return "go buy"
    else:
        return "wrong go buy"

def sell_quality(hi,lo,clo,start,sl):
    if sl == 'sell' and abs(min(lo[1:]) - start) > abs(max(hi[1:]) - start):
        return "go sell"
    else:
        return "wrong go sell"

def mover(hi,lo,clo,start):
    up_move = max(hi[1:]) - start
    low_move = start - min(lo[1:])
    return up_move, low_move

def buy_calc_side(hi,lo,clo,start):
    if statistics.median(clo[1:]) > start:
        return "buy"
    else:
        return "wrong buy"

def sell_calc_side(hi,lo,clo,start):
    if statistics.median(clo[1:]) < start:
        return "sell"
    else:
        return "wrong sell"
    
def label_calc(df):
    buy_lab = []
    sell_lab = []
    bq = []
    sq = []
    go = []
    profit = []
    loss = []
    k = 10
    for t in range(0,len(df)):
        if t >= len(df) - k:
            break
        a1 = df[t:t+k]
        a0 = df[t:t+k]
        st = a0.Close.tolist()[0]
        hi = a1.High.tolist()
        lo = a1.Low.tolist()
        clo = a1.Close.tolist()
        buy_label = buy_calc_side(hi,lo,clo,st)
        sell_label = sell_calc_side(hi,lo,clo,st)
        b_quality = buy_quality(hi,lo,clo,st,buy_label)
        s_quality = sell_quality(hi,lo,clo,st,sell_label)
        up_move, low_move = mover(hi,lo,clo,st)
        
        
        buy_lab += [buy_label]
        sell_lab += [sell_label]
        bq += [b_quality]
        sq += [s_quality]
        
        
        profit += [up_move]
        loss += [low_move]

    if len(buy_lab) < len(df):
        buy_lab += ["Pass"] * (len(df)-len(buy_lab))
    if len(sell_lab) < len(df):
        sell_lab += ["Pass"] * (len(df)-len(sell_lab))
    if len(sq) < len(df):
        sq += ["Pass"] * (len(df)-len(sq))
    if len(bq) < len(df):
        bq += ["Pass"] * (len(df)-len(bq))
    if len(profit) < len(df):
        profit += [0] * (len(df)-len(profit))
    if len(loss) < len(df):
        loss += [0] * (len(df)-len(loss))

    df["buy_label"] = buy_lab
    df["sell_label"] = sell_lab
    df["bu_quality"] = bq
    df["se_quality"] = sq
    df['Profit'] = profit
    df['Loss'] = loss
    

    return df

In [5]:
def df_vectorize(df):
    vec_load = []
    m5_vec = []
    buy_side = []
    sell_side = []
    b_quality = []
    s_quality = []
    take_profit = []
    stop_loss = []
    print(len(df))
    for h in range(52,len(df)):
        buff_df = df[h-52:h]
        if buff_df.buy_label.tolist()[-1] != "Skip":
            
            orig_close = buff_df.cl.tolist()
            orig_hi = buff_df.hi.tolist()
            orig_lo = buff_df.lo.tolist()
            orig_op = buff_df.op.tolist()
            close_log = buff_df.Close.tolist()
            hi = buff_df.High.tolist()
            lo = buff_df.Low.tolist()
            op = buff_df.Open.tolist()
            rb_vec = buff_df.rb.tolist()
            us_vec = buff_df.us.tolist()
            ls_vec = buff_df.ls.tolist()
            pct = buff_df.pct.tolist()
            t_ma = buff_df['20_SMA'].tolist()
            f_ma = buff_df['5_EMA'].tolist()
            sma = buff_df.sma.tolist()
            fema = buff_df.fema.tolist()
            atr = buff_df.ATR.tolist()
            ma_diff = buff_df.ma_diff.tolist()
            tclose_diff = buff_df.tclose_diff.tolist()
            fclose_diff = buff_df.fclose_diff.tolist()
            med = statistics.median(buff_df.cl.tolist())
            maximum = max(buff_df.cl.tolist())
            minimum = min(buff_df.cl.tolist())
            stand_dev = statistics.stdev(buff_df.cl.tolist())
            
            buy_side += [buff_df.buy_label.tolist()[-1]]
            sell_side += [buff_df.sell_label.tolist()[-1]]
            b_quality += [buff_df.bu_quality.tolist()[-1]]
            s_quality += [buff_df.se_quality.tolist()[-1]]
            take_profit += [buff_df.Profit.tolist()[-1]]
            stop_loss += [buff_df.Loss.tolist()[-1]]

            #vec_load += [close_log + rb_vec + us_vec + ls_vec + ma_diff + tclose_diff + fclose_diff]
            m5_vec += [rb_vec + us_vec + ls_vec + ma_diff + atr + pct]

        else:
            print('Skip')
    print('Vector_Done')
    vec_load = m5_vec
    return vec_load, buy_side, sell_side, b_quality, s_quality, take_profit, stop_loss
        
        

In [6]:
from numpy_ext import rolling_apply

In [7]:
from fracdiff import FracdiffStat
cf = FracdiffStat()
hf = FracdiffStat()
of = FracdiffStat()
lf = FracdiffStat()
mf = FracdiffStat()


In [8]:
#df = pd.read_csv('SPX_5min_last_3.csv')
#df = df[~((df.Close == df.High) & (df.Close == df.Low) & (df.Close == df.Open))]
df = pd.read_csv('last_10_train.csv')
df.Close = list(reversed(df.Close.tolist()))
df.Open = list(reversed(df.Open.tolist()))
df.High = list(reversed(df.High.tolist()))
df.Low = list(reversed(df.Low.tolist()))
df = label_calc(df)
df = indicator_fill(df)
df = df[20:]
df = df[df.buy_label!='Pass']
print('label done')
df['cl'] = df.Close
df['pct'] = df.Close.pct_change()
df['hi'] = df.High
df['lo'] = df.Low
df['op'] = df.Open
df['sma'] = df['20_SMA']
df['fema'] = df['5_EMA']

df['us'] = df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
df['ls'] = df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
df['rb'] = df.apply(lambda x: real_body(x.Open, x.Close), axis = 1)
df['ma_diff'] = df['5_EMA'] - df['20_SMA']
df['tclose_diff'] = df['Close'] - df['20_SMA']
df['fclose_diff'] = df['Close'] - df['5_EMA']
print('closefracdiff')
df.Close = list(np.squeeze(cf.fit_transform(np.array(df.Close).reshape(-1,1)), axis = 1))
print('hifracdiff')
df.High = list(np.squeeze(cf.transform(np.array(df.High).reshape(-1,1)), axis = 1))
print('openfracdiff')
df.Open = list(np.squeeze(cf.transform(np.array(df.Open).reshape(-1,1)), axis = 1))
print('lowfracdiff')
df.Low = list(np.squeeze(cf.transform(np.array(df.Low).reshape(-1,1)), axis = 1))
print('tema')
df['20_SMA'] = list(np.squeeze(cf.transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
print('fema')
df['5_EMA'] = list(np.squeeze(cf.transform(np.array(df['5_EMA']).reshape(-1,1)), axis = 1))
df = df[1:]
v,bs,ss,bq_vec,sq_vec,tp,sl = df_vectorize(df)


label done
closefracdiff
hifracdiff
openfracdiff
lowfracdiff
tema
fema
2347
Vector_Done


In [11]:
df = pd.read_csv('last_10_test.csv')
df.Close = list(reversed(df.Close.tolist()))
df.Open = list(reversed(df.Open.tolist()))
df.High = list(reversed(df.High.tolist()))
df.Low = list(reversed(df.Low.tolist()))
df = label_calc(df)
df = indicator_fill(df)
print(len(df))
df = df[20:]
print(len(df))
df = df[df.buy_label!='Pass']

df['cl'] = df.Close
df['pct'] = df.Close.pct_change()
df['hi'] = df.High
df['lo'] = df.Low
df['op'] = df.Open
df['sma'] = df['20_SMA']
df['fema'] = df['5_EMA']


df['ma_diff'] = df['5_EMA'] - df['20_SMA']
df['tclose_diff'] = df['Close'] - df['20_SMA']
df['fclose_diff'] = df['Close'] - df['5_EMA']
df.Close = list(np.squeeze(cf.fit_transform(np.array(df.Close).reshape(-1,1)), axis = 1))
df.High = list(np.squeeze(cf.transform(np.array(df.High).reshape(-1,1)), axis = 1))
df.Open = list(np.squeeze(cf.transform(np.array(df.Open).reshape(-1,1)), axis = 1))
df.Low = list(np.squeeze(cf.transform(np.array(df.Low).reshape(-1,1)), axis = 1))
df['20_SMA'] = list(np.squeeze(cf.transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
df['5_EMA'] = list(np.squeeze(cf.transform(np.array(df['5_EMA']).reshape(-1,1)), axis = 1))

#df['20_SMA'] = list(np.squeeze(cf.transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
df = df[1:]
print(len(df))

df['us'] = df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
df['ls'] = df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
df['rb'] = df.apply(lambda x: real_body(x.Open, x.Close), axis = 1)
vt,bst,sst,bqt,sqt,tpt,slt = df_vectorize(df)
df = df[10:]
vt = np.array(vt)

bside_lab = buy_le.transform(bst)
sside_lab = sell_le.transform(sst)
bqside_lab = bq_le.transform(bqt)
sqside_lab = sq_le.transform(sqt)


180
160
149
149
Vector_Done


In [63]:
print(statistics.median(tp))
print(statistics.median(sl))
print(max(tp))
print(max(sl))

30.819999999999936
31.570000000000164
403.9899999999998
2130.82


In [10]:
import numpy as np
import pickle
from joblib import dump,load

v = np.array(v)
v = np.nan_to_num(v)
# D candle best feature selection based on ANOVA and fregression

tp = np.array(tp).reshape(-1,1)
sl = np.array(sl).reshape(-1,1)
#v_h = np.hstack((v,tp,sl))
from sklearn.preprocessing import LabelEncoder

buy_le = LabelEncoder()
buy_lab = buy_le.fit_transform(bs)
sell_le = LabelEncoder()
sell_lab = sell_le.fit_transform(ss)

sq_le = LabelEncoder()
sq_lab = sq_le.fit_transform(sq_vec)
bq_le = LabelEncoder()
bq_lab = bq_le.fit_transform(bq_vec)


dump(buy_le,'buy_le_SPX.joblib')
dump(sell_le,'sell_le_SPX.joblib')
dump(bq_le,'bq_le_SPX.joblib')
dump(sq_le,'sq_le_SPX.joblib')

['sq_le_SPX.joblib']

In [10]:
from collections import Counter

print(Counter(buy_lab))
print(Counter(sell_lab))
print(Counter(bq_lab))
print(Counter(sq_lab))

Counter({0: 1462, 1: 833})
Counter({1: 1464, 0: 831})
Counter({1: 1214, 0: 1081})
Counter({1: 1494, 0: 801})


In [11]:
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NearMiss
ov1 = SMOTE()
undersample = NearMiss(version=3, n_neighbors_ver3=3)
v_v,buy_lab= undersample.fit_resample(v,buy_lab)
print(Counter(buy_lab))


Counter({0: 833, 1: 833})


In [12]:
v_s,sell_lab= undersample.fit_resample(v,sell_lab)
#v_bq,bq_lab= undersample.fit_resample(v,bq_lab)
v_sq,sq_lab= undersample.fit_resample(v,sq_lab)
print(Counter(sell_lab))
print(Counter(bq_lab))
print(Counter(sq_lab))

Counter({0: 831, 1: 831})
Counter({1: 1214, 0: 1081})
Counter({0: 801, 1: 801})


In [68]:
from sklearn.model_selection import train_test_split
X_trainb, X_testb, y_trainb, y_testb = train_test_split(v_v, buy_lab, test_size=0.1, random_state=42, stratify=buy_lab)
X_trains, X_tests, y_trains, y_tests = train_test_split(v_s, sell_lab, test_size=0.1, random_state=42, stratify=sell_lab)
X_trainbq, X_testbq, y_trainbq, y_testbq = train_test_split(v, bq_lab, test_size=0.1, random_state=42, stratify=bq_lab)
X_trainsq, X_testsq, y_trainsq, y_testsq = train_test_split(v_sq, sq_lab, test_size=0.1, random_state=42, stratify=sq_lab)
X_train_tp, X_test_tp, y_train_tp, y_test_tp = train_test_split(v, tp, test_size=0.1, random_state=42)
X_train_sl, X_test_sl, y_train_sl, y_test_sl = train_test_split(v, sl, test_size=0.1, random_state=42)
print(X_trainb.shape)
print(y_trainb.shape)
print(X_train_tp.shape)
print(y_train_tp.shape)
print(X_train_sl.shape)
print(y_train_sl.shape)

(1535, 70)
(1535,)
(2103, 70)
(2103, 1)
(2103, 70)
(2103, 1)


# Keras GRU Training

## Buy_model

In [13]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience = 100)
mc = ModelCheckpoint('best_bi-GRU_SPX_buy_model_with_stats_RealVal.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
dim = 70
v = v.reshape(-1,52,6)
vt = vt.reshape(-1,52,6)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU, Dense, Bidirectional, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.constraints import max_norm
buy_model = Sequential()
buy_model.add(Dropout(0.2))
buy_model.add(Bidirectional(GRU(128, input_shape=(52,6), return_sequences = True, kernel_constraint = max_norm(3.))))
buy_model.add(Dropout(0.5))
buy_model.add(BatchNormalization())
buy_model.add(Bidirectional(GRU(128, return_sequences = True, kernel_constraint = max_norm(3.))))
buy_model.add(Dropout(0.5))
buy_model.add(BatchNormalization())
buy_model.add(Bidirectional(GRU(64, kernel_constraint = max_norm(3.))))
buy_model.add(Dropout(0.5))
buy_model.add(BatchNormalization())
buy_model.add(Dense(32, activation='relu', kernel_constraint = max_norm(3.)))
buy_model.add(Dropout(0.5))
buy_model.add(BatchNormalization())
buy_model.add(Dense(2, activation='sigmoid'))
buy_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
buy_model.fit(v,buy_lab, epochs=250, batch_size=128, verbose=1, use_multiprocessing=True, 
          validation_data=(vt,bside_lab),
         callbacks=[mc])

Epoch 1/250
18/18 [==============================] - 19s 508ms/step - loss: 1.0104 - accuracy: 0.5150 - val_loss: 0.6830 - val_accuracy: 0.5979
Epoch 2/250
18/18 [==============================] - 8s 421ms/step - loss: 0.9316 - accuracy: 0.5242 - val_loss: 0.7197 - val_accuracy: 0.5155
Epoch 3/250
18/18 [==============================] - 8s 449ms/step - loss: 0.8838 - accuracy: 0.5420 - val_loss: 0.7132 - val_accuracy: 0.4330
Epoch 4/250
18/18 [==============================] - 8s 441ms/step - loss: 0.8533 - accuracy: 0.5564 - val_loss: 0.7039 - val_accuracy: 0.4536
Epoch 5/250
18/18 [==============================] - 8s 447ms/step - loss: 0.8311 - accuracy: 0.5538 - val_loss: 0.7209 - val_accuracy: 0.4330
Epoch 6/250
18/18 [==============================] - 8s 447ms/step - loss: 0.7950 - accuracy: 0.5630 - val_loss: 0.6890 - val_accuracy: 0.5155
Epoch 7/250
18/18 [==============================] - 8s 462ms/step - loss: 0.7744 - accuracy: 0.5669 - val_loss: 0.6578 - val_accuracy: 0.659

Epoch 58/250
18/18 [==============================] - 9s 485ms/step - loss: 0.6464 - accuracy: 0.6366 - val_loss: 0.6226 - val_accuracy: 0.6907
Epoch 59/250
18/18 [==============================] - 9s 483ms/step - loss: 0.6464 - accuracy: 0.6401 - val_loss: 0.6375 - val_accuracy: 0.6907
Epoch 60/250
18/18 [==============================] - 9s 481ms/step - loss: 0.6459 - accuracy: 0.6397 - val_loss: 0.6349 - val_accuracy: 0.6907
Epoch 61/250
18/18 [==============================] - 9s 484ms/step - loss: 0.6478 - accuracy: 0.6331 - val_loss: 0.6429 - val_accuracy: 0.6804
Epoch 62/250
18/18 [==============================] - 9s 482ms/step - loss: 0.6394 - accuracy: 0.6357 - val_loss: 0.6456 - val_accuracy: 0.6907
Epoch 63/250
18/18 [==============================] - 9s 482ms/step - loss: 0.6429 - accuracy: 0.6383 - val_loss: 0.6450 - val_accuracy: 0.6907
Epoch 64/250
18/18 [==============================] - 9s 480ms/step - loss: 0.6497 - accuracy: 0.6340 - val_loss: 0.6197 - val_accuracy:

Epoch 115/250
18/18 [==============================] - 9s 482ms/step - loss: 0.5440 - accuracy: 0.7111 - val_loss: 0.7463 - val_accuracy: 0.5052
Epoch 116/250
18/18 [==============================] - 9s 486ms/step - loss: 0.5388 - accuracy: 0.7316 - val_loss: 0.7224 - val_accuracy: 0.6186
Epoch 117/250
18/18 [==============================] - 9s 490ms/step - loss: 0.5526 - accuracy: 0.7172 - val_loss: 0.8975 - val_accuracy: 0.3608
Epoch 118/250
18/18 [==============================] - 9s 489ms/step - loss: 0.5468 - accuracy: 0.7120 - val_loss: 0.6559 - val_accuracy: 0.5670
Epoch 119/250
18/18 [==============================] - 9s 498ms/step - loss: 0.5424 - accuracy: 0.7285 - val_loss: 0.7173 - val_accuracy: 0.6701
Epoch 120/250
18/18 [==============================] - 9s 475ms/step - loss: 0.5350 - accuracy: 0.7255 - val_loss: 0.6951 - val_accuracy: 0.5464
Epoch 121/250
18/18 [==============================] - 8s 468ms/step - loss: 0.5242 - accuracy: 0.7394 - val_loss: 0.7852 - val_ac

18/18 [==============================] - 8s 468ms/step - loss: 0.4195 - accuracy: 0.8096 - val_loss: 0.8566 - val_accuracy: 0.5876
Epoch 172/250
18/18 [==============================] - 8s 467ms/step - loss: 0.4220 - accuracy: 0.8083 - val_loss: 0.7385 - val_accuracy: 0.5876
Epoch 173/250
18/18 [==============================] - 8s 470ms/step - loss: 0.4146 - accuracy: 0.8131 - val_loss: 0.7263 - val_accuracy: 0.5979
Epoch 174/250
18/18 [==============================] - 8s 469ms/step - loss: 0.4141 - accuracy: 0.8118 - val_loss: 0.8487 - val_accuracy: 0.5361
Epoch 175/250
18/18 [==============================] - 8s 469ms/step - loss: 0.4237 - accuracy: 0.8096 - val_loss: 0.8660 - val_accuracy: 0.5773
Epoch 176/250
18/18 [==============================] - 8s 469ms/step - loss: 0.4050 - accuracy: 0.8100 - val_loss: 0.8667 - val_accuracy: 0.5258
Epoch 177/250
18/18 [==============================] - 8s 464ms/step - loss: 0.3950 - accuracy: 0.8283 - val_loss: 0.7356 - val_accuracy: 0.5979

18/18 [==============================] - 8s 466ms/step - loss: 0.3276 - accuracy: 0.8632 - val_loss: 0.9346 - val_accuracy: 0.5567
Epoch 228/250
18/18 [==============================] - 8s 465ms/step - loss: 0.3161 - accuracy: 0.8641 - val_loss: 1.1652 - val_accuracy: 0.5670
Epoch 229/250
18/18 [==============================] - 8s 468ms/step - loss: 0.3370 - accuracy: 0.8527 - val_loss: 0.8820 - val_accuracy: 0.6289
Epoch 230/250
18/18 [==============================] - 8s 466ms/step - loss: 0.3235 - accuracy: 0.8562 - val_loss: 0.8868 - val_accuracy: 0.6392
Epoch 231/250
18/18 [==============================] - 8s 467ms/step - loss: 0.3195 - accuracy: 0.8606 - val_loss: 1.1999 - val_accuracy: 0.4948
Epoch 232/250
18/18 [==============================] - 8s 466ms/step - loss: 0.3194 - accuracy: 0.8641 - val_loss: 1.3348 - val_accuracy: 0.4433
Epoch 233/250
18/18 [==============================] - 8s 464ms/step - loss: 0.3455 - accuracy: 0.8514 - val_loss: 0.9010 - val_accuracy: 0.5052

In [14]:
from tensorflow.keras.models import load_model
bmodel = load_model('best_bi-GRU_SPX_buy_model_with_stats_RealVal.h5')
pp = np.argmax(bmodel.predict(vt), axis=1)
kk = np.max(bmodel.predict(vt), axis=1)
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
print('Buy Side--------------------------')
print(accuracy_score(bside_lab, pp))
print(classification_report(bside_lab, pp))


Buy Side--------------------------
0.7010309278350515
              precision    recall  f1-score   support

           0       0.70      1.00      0.82        67
           1       1.00      0.03      0.06        30

    accuracy                           0.70        97
   macro avg       0.85      0.52      0.44        97
weighted avg       0.79      0.70      0.59        97



In [15]:
print(pp)
print(bside_lab)
print(kk)

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 1 1 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 1]
[0.6122777  0.61284125 0.63208896 0.6793514  0.6215278  0.58885115
 0.60562634 0.63727355 0.604082   0.62207586 0.6146509  0.6380168
 0.5756972  0.5995049  0.63096106 0.64049387 0.57765913 0.6215456
 0.5972481  0.6066892  0.5835854  0.6264858  0.59853643 0.58289987
 0.6448701  0.55524176 0.5651584  0.666909   0.6384873  0.6286603
 0.58591795 0.5809804  0.52967006 0.66460127 0.6107118  0.66742927
 0.5720729  0.56973135 0.55168545 0.60111845 0.5721045  0.66831434
 0.5315737  0.64738953 0.6247201  0.5496865  0.58937895 0.6599535
 0.5247733  0.6658948  0.60822964 0.63853157 0.63929266 0.6437035
 0.570

## Sell Model

In [60]:
mcs = ModelCheckpoint('best_bi-GRU_SPX_sell_model_with_stats_RealVal.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
sell_model = Sequential()
sell_model.add(Bidirectional(GRU(128, input_shape=(100,7), return_sequences = True)))
sell_model.add(Dropout(0.2))
sell_model.add(BatchNormalization())
sell_model.add(GRU(64, return_sequences = True))
sell_model.add(Dropout(0.2))
sell_model.add(BatchNormalization())
sell_model.add(GRU(32))
sell_model.add(Dropout(0.2))
sell_model.add(BatchNormalization())
sell_model.add(Dense(16, activation='relu'))
sell_model.add(Dropout(0.2))
sell_model.add(BatchNormalization())
sell_model.add(Dense(2, activation='sigmoid'))
sell_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
sell_model.fit(X_trains_gru, y_trains, epochs=250, batch_size=128, verbose=2, use_multiprocessing=True, 
          validation_data=(X_tests_gru, y_tests),
         callbacks=[mcs])

Epoch 1/250
12/12 - 13s - loss: 0.9053 - accuracy: 0.4945 - val_loss: 0.6910 - val_accuracy: 0.5247
Epoch 2/250
12/12 - 5s - loss: 0.7545 - accuracy: 0.5731 - val_loss: 0.6952 - val_accuracy: 0.5432
Epoch 3/250
12/12 - 5s - loss: 0.6904 - accuracy: 0.5917 - val_loss: 0.6985 - val_accuracy: 0.5556
Epoch 4/250
12/12 - 5s - loss: 0.7243 - accuracy: 0.5752 - val_loss: 0.6981 - val_accuracy: 0.5247
Epoch 5/250
12/12 - 5s - loss: 0.6938 - accuracy: 0.5959 - val_loss: 0.6912 - val_accuracy: 0.5432
Epoch 6/250
12/12 - 5s - loss: 0.6762 - accuracy: 0.6090 - val_loss: 0.6843 - val_accuracy: 0.5802
Epoch 7/250
12/12 - 5s - loss: 0.6323 - accuracy: 0.6510 - val_loss: 0.6891 - val_accuracy: 0.5123
Epoch 8/250
12/12 - 5s - loss: 0.6521 - accuracy: 0.6248 - val_loss: 0.6870 - val_accuracy: 0.5309
Epoch 9/250
12/12 - 5s - loss: 0.6173 - accuracy: 0.6600 - val_loss: 0.6927 - val_accuracy: 0.5247
Epoch 10/250
12/12 - 5s - loss: 0.5926 - accuracy: 0.6641 - val_loss: 0.6885 - val_accuracy: 0.5432
Epoch 11

12/12 - 5s - loss: 0.0517 - accuracy: 0.9848 - val_loss: 1.9738 - val_accuracy: 0.6481
Epoch 84/250
12/12 - 6s - loss: 0.0969 - accuracy: 0.9724 - val_loss: 1.9288 - val_accuracy: 0.6543
Epoch 85/250
12/12 - 6s - loss: 0.0915 - accuracy: 0.9738 - val_loss: 1.9557 - val_accuracy: 0.5926
Epoch 86/250
12/12 - 6s - loss: 0.0729 - accuracy: 0.9800 - val_loss: 1.9582 - val_accuracy: 0.6481
Epoch 87/250
12/12 - 5s - loss: 0.0602 - accuracy: 0.9738 - val_loss: 1.7724 - val_accuracy: 0.6481
Epoch 88/250
12/12 - 5s - loss: 0.0484 - accuracy: 0.9834 - val_loss: 1.8315 - val_accuracy: 0.6481
Epoch 89/250
12/12 - 5s - loss: 0.0432 - accuracy: 0.9869 - val_loss: 1.9844 - val_accuracy: 0.6543
Epoch 90/250
12/12 - 5s - loss: 0.0325 - accuracy: 0.9924 - val_loss: 2.1113 - val_accuracy: 0.6049
Epoch 91/250
12/12 - 5s - loss: 0.0335 - accuracy: 0.9924 - val_loss: 2.0078 - val_accuracy: 0.6481
Epoch 92/250
12/12 - 5s - loss: 0.0274 - accuracy: 0.9910 - val_loss: 2.1536 - val_accuracy: 0.6358
Epoch 93/250


Epoch 165/250


KeyboardInterrupt: 

## Buy Quality Model (Meta)

In [106]:
mcbq = ModelCheckpoint('best_bi-GRU_SPX_buy-quality_model_with_stats_RealVal.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
bq_model = Sequential()
bq_model.add(GRU(128, input_shape=(X_trainbq_gru.shape[1],1), return_sequences = True))
bq_model.add(Dropout(0.2))
bq_model.add(GRU(64, return_sequences = True))
bq_model.add(Dropout(0.2))
bq_model.add(BatchNormalization())
bq_model.add(GRU(32))
bq_model.add(Dropout(0.2))
bq_model.add(BatchNormalization())
bq_model.add(Dense(16, activation='relu'))
bq_model.add(Dropout(0.2))
bq_model.add(BatchNormalization())
bq_model.add(Dense(2, activation='sigmoid'))
bq_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
bq_model.fit(X_trainbq_gru, y_trainbq, epochs=100, batch_size=128, verbose=2, use_multiprocessing=True, 
          validation_data=(X_testbq_gru, y_testbq),
         callbacks=[es, mcbq])

Epoch 1/100
17/17 - 6s - loss: 0.9165 - accuracy: 0.5231 - val_loss: 0.6956 - val_accuracy: 0.4957
Epoch 2/100
17/17 - 0s - loss: 0.8524 - accuracy: 0.5107 - val_loss: 0.6875 - val_accuracy: 0.5726
Epoch 3/100
17/17 - 0s - loss: 0.8266 - accuracy: 0.5159 - val_loss: 0.6795 - val_accuracy: 0.6026
Epoch 4/100
17/17 - 0s - loss: 0.7920 - accuracy: 0.5288 - val_loss: 0.6829 - val_accuracy: 0.5983
Epoch 5/100
17/17 - 0s - loss: 0.7740 - accuracy: 0.5226 - val_loss: 0.6783 - val_accuracy: 0.5983
Epoch 6/100
17/17 - 0s - loss: 0.7762 - accuracy: 0.5426 - val_loss: 0.6806 - val_accuracy: 0.5726
Epoch 7/100
17/17 - 0s - loss: 0.7516 - accuracy: 0.5411 - val_loss: 0.6768 - val_accuracy: 0.5769
Epoch 8/100
17/17 - 0s - loss: 0.7452 - accuracy: 0.5345 - val_loss: 0.6757 - val_accuracy: 0.5726
Epoch 9/100
17/17 - 0s - loss: 0.7232 - accuracy: 0.5635 - val_loss: 0.6766 - val_accuracy: 0.5769
Epoch 10/100
17/17 - 0s - loss: 0.7398 - accuracy: 0.5354 - val_loss: 0.6760 - val_accuracy: 0.5769
Epoch 11/

17/17 - 0s - loss: 0.5865 - accuracy: 0.6786 - val_loss: 0.6306 - val_accuracy: 0.6282
Epoch 84/100
17/17 - 0s - loss: 0.5771 - accuracy: 0.6762 - val_loss: 0.6499 - val_accuracy: 0.6581
Epoch 85/100
17/17 - 0s - loss: 0.5758 - accuracy: 0.6805 - val_loss: 0.6332 - val_accuracy: 0.6368
Epoch 86/100
17/17 - 0s - loss: 0.5781 - accuracy: 0.6814 - val_loss: 0.6450 - val_accuracy: 0.6111
Epoch 87/100
17/17 - 0s - loss: 0.5672 - accuracy: 0.6890 - val_loss: 0.6255 - val_accuracy: 0.6538
Epoch 88/100
17/17 - 0s - loss: 0.5779 - accuracy: 0.6857 - val_loss: 0.6439 - val_accuracy: 0.6197
Epoch 89/100
17/17 - 0s - loss: 0.5739 - accuracy: 0.6824 - val_loss: 0.6462 - val_accuracy: 0.5897
Epoch 90/100
17/17 - 0s - loss: 0.5696 - accuracy: 0.6890 - val_loss: 0.6438 - val_accuracy: 0.6496
Epoch 91/100
17/17 - 0s - loss: 0.5666 - accuracy: 0.6857 - val_loss: 0.6419 - val_accuracy: 0.6282
Epoch 92/100
17/17 - 0s - loss: 0.5603 - accuracy: 0.6919 - val_loss: 0.6623 - val_accuracy: 0.6325
Epoch 93/100


## Sell Quality Model (Meta)

In [107]:
mcsq = ModelCheckpoint('best_bi-GRU_SPX_sell-quality_model_with_stats_RealVal.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
sq_model = Sequential()
sq_model.add(GRU(128, input_shape=(X_trainsq_gru.shape[1],1), return_sequences = True))
sq_model.add(Dropout(0.2))
sq_model.add(GRU(64, return_sequences = True))
sq_model.add(Dropout(0.2))
sq_model.add(BatchNormalization())
sq_model.add(GRU(32))
sq_model.add(Dropout(0.2))
sq_model.add(BatchNormalization())
sq_model.add(Dense(16, activation='relu'))
sq_model.add(Dropout(0.2))
sq_model.add(BatchNormalization())
sq_model.add(Dense(2, activation='sigmoid'))
sq_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
sq_model.fit(X_trainsq_gru, y_trainsq, epochs=100, batch_size=128, verbose=2, use_multiprocessing=True, 
          validation_data=(X_testsq_gru, y_testsq),
         callbacks=[es, mcsq])

Epoch 1/100
12/12 - 7s - loss: 0.9245 - accuracy: 0.5318 - val_loss: 0.7111 - val_accuracy: 0.4756
Epoch 2/100
12/12 - 0s - loss: 0.8689 - accuracy: 0.5014 - val_loss: 0.7017 - val_accuracy: 0.4573
Epoch 3/100
12/12 - 0s - loss: 0.8556 - accuracy: 0.5210 - val_loss: 0.6982 - val_accuracy: 0.5366
Epoch 4/100
12/12 - 0s - loss: 0.8194 - accuracy: 0.5325 - val_loss: 0.6962 - val_accuracy: 0.5061
Epoch 5/100
12/12 - 0s - loss: 0.8059 - accuracy: 0.5495 - val_loss: 0.7001 - val_accuracy: 0.5183
Epoch 6/100
12/12 - 0s - loss: 0.7850 - accuracy: 0.5501 - val_loss: 0.6948 - val_accuracy: 0.5549
Epoch 7/100
12/12 - 0s - loss: 0.7550 - accuracy: 0.5488 - val_loss: 0.6907 - val_accuracy: 0.5671
Epoch 8/100
12/12 - 0s - loss: 0.7762 - accuracy: 0.5495 - val_loss: 0.6925 - val_accuracy: 0.5854
Epoch 9/100
12/12 - 0s - loss: 0.7804 - accuracy: 0.5339 - val_loss: 0.6903 - val_accuracy: 0.6037
Epoch 10/100
12/12 - 0s - loss: 0.7634 - accuracy: 0.5454 - val_loss: 0.6835 - val_accuracy: 0.5610
Epoch 11/

12/12 - 0s - loss: 0.6119 - accuracy: 0.6491 - val_loss: 0.6384 - val_accuracy: 0.6098
Epoch 84/100
12/12 - 0s - loss: 0.6100 - accuracy: 0.6396 - val_loss: 0.6405 - val_accuracy: 0.5976
Epoch 85/100
12/12 - 0s - loss: 0.6211 - accuracy: 0.6457 - val_loss: 0.6280 - val_accuracy: 0.6280
Epoch 86/100
12/12 - 0s - loss: 0.6166 - accuracy: 0.6470 - val_loss: 0.6334 - val_accuracy: 0.6220
Epoch 87/100
12/12 - 0s - loss: 0.6137 - accuracy: 0.6511 - val_loss: 0.6339 - val_accuracy: 0.6159
Epoch 88/100
12/12 - 0s - loss: 0.6086 - accuracy: 0.6402 - val_loss: 0.6334 - val_accuracy: 0.6037
Epoch 89/100
12/12 - 0s - loss: 0.6024 - accuracy: 0.6524 - val_loss: 0.6219 - val_accuracy: 0.6524
Epoch 90/100
12/12 - 0s - loss: 0.6040 - accuracy: 0.6565 - val_loss: 0.6253 - val_accuracy: 0.6037
Epoch 91/100
12/12 - 0s - loss: 0.6013 - accuracy: 0.6646 - val_loss: 0.6378 - val_accuracy: 0.6159
Epoch 92/100
12/12 - 0s - loss: 0.6086 - accuracy: 0.6450 - val_loss: 0.6229 - val_accuracy: 0.6098
Epoch 93/100


## Price Movement Regression

In [108]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience = 100)


from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU, Dense, Bidirectional, Dropout
from tensorflow.keras.layers import BatchNormalization
mctp = ModelCheckpoint('best_bi-GRU_SPX_Price-Up-Move_model_with_stats_RealVal.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
X_traintp_gru = X_train_tp.reshape(-1,X_train_tp.shape[1],1)
X_testtp_gru = X_test_tp.reshape(-1,X_test_tp.shape[1],1)
tp_model = Sequential()
tp_model.add(Bidirectional(GRU(128, input_shape=(X_train_tp.shape[1],1), return_sequences = True)))
tp_model.add(Dropout(0.2))
tp_model.add(GRU(64, return_sequences = True))
tp_model.add(Dropout(0.2))
tp_model.add(BatchNormalization())
tp_model.add(GRU(32))
tp_model.add(Dropout(0.2))
tp_model.add(BatchNormalization())
tp_model.add(Dense(16, activation='relu'))
tp_model.add(Dropout(0.2))
tp_model.add(BatchNormalization())
tp_model.add(Dense(1))
tp_model.compile(loss='mae', optimizer='adam')
tp_model.fit(X_traintp_gru, y_train_tp, epochs=200, batch_size=128, verbose=2, use_multiprocessing=True, 
          validation_data=(X_testtp_gru, y_test_tp),
         callbacks=[es, mctp])

Epoch 1/200
17/17 - 8s - loss: 41.4729 - val_loss: 44.3895
Epoch 2/200
17/17 - 1s - loss: 41.3494 - val_loss: 44.3221
Epoch 3/200
17/17 - 1s - loss: 41.2457 - val_loss: 43.9198
Epoch 4/200
17/17 - 1s - loss: 41.1383 - val_loss: 43.9481
Epoch 5/200
17/17 - 1s - loss: 40.9891 - val_loss: 43.9348
Epoch 6/200
17/17 - 1s - loss: 40.8694 - val_loss: 43.4636
Epoch 7/200
17/17 - 1s - loss: 40.7141 - val_loss: 43.7131
Epoch 8/200
17/17 - 1s - loss: 40.5197 - val_loss: 43.2691
Epoch 9/200
17/17 - 1s - loss: 40.3422 - val_loss: 43.2422
Epoch 10/200
17/17 - 1s - loss: 40.1020 - val_loss: 42.8433
Epoch 11/200
17/17 - 1s - loss: 39.8743 - val_loss: 42.1852
Epoch 12/200
17/17 - 1s - loss: 39.6691 - val_loss: 41.5321
Epoch 13/200
17/17 - 1s - loss: 39.3603 - val_loss: 41.2531
Epoch 14/200
17/17 - 1s - loss: 39.0608 - val_loss: 42.8333
Epoch 15/200
17/17 - 1s - loss: 38.8235 - val_loss: 42.4305
Epoch 16/200
17/17 - 1s - loss: 38.4448 - val_loss: 40.7436
Epoch 17/200
17/17 - 1s - loss: 38.0854 - val_los

17/17 - 1s - loss: 14.7747 - val_loss: 21.0655
Epoch 138/200
17/17 - 1s - loss: 14.8787 - val_loss: 20.2887
Epoch 139/200
17/17 - 1s - loss: 14.8275 - val_loss: 20.3505
Epoch 140/200
17/17 - 1s - loss: 14.6210 - val_loss: 20.4417
Epoch 141/200
17/17 - 1s - loss: 14.6848 - val_loss: 20.0233
Epoch 142/200
17/17 - 1s - loss: 14.3750 - val_loss: 20.8136
Epoch 143/200
17/17 - 1s - loss: 14.7809 - val_loss: 20.0406
Epoch 144/200
17/17 - 1s - loss: 14.2019 - val_loss: 20.4669
Epoch 145/200
17/17 - 1s - loss: 14.6583 - val_loss: 19.6098
Epoch 146/200
17/17 - 1s - loss: 14.5015 - val_loss: 20.1712
Epoch 147/200
17/17 - 1s - loss: 14.2476 - val_loss: 19.9084
Epoch 148/200
17/17 - 1s - loss: 14.0915 - val_loss: 19.3136
Epoch 149/200
17/17 - 1s - loss: 14.2595 - val_loss: 20.2822
Epoch 150/200
17/17 - 1s - loss: 14.6401 - val_loss: 20.0752
Epoch 151/200
17/17 - 1s - loss: 14.1246 - val_loss: 20.4095
Epoch 152/200
17/17 - 1s - loss: 14.2086 - val_loss: 19.9393
Epoch 153/200
17/17 - 1s - loss: 14.06

In [109]:
mcsl = ModelCheckpoint('best_bi-GRU_SPX_Price-Down-Move_model_with_stats_RealVal.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)
X_trainsl_gru = X_train_sl.reshape(-1,X_train_sl.shape[1],1)
X_testsl_gru = X_test_sl.reshape(-1,X_test_sl.shape[1],1)
sl_model = Sequential()
sl_model.add(Bidirectional(GRU(128, input_shape=(X_train_sl.shape[1],1), return_sequences = True)))
sl_model.add(Dropout(0.2))
sl_model.add(GRU(64, return_sequences = True))
sl_model.add(Dropout(0.2))
sl_model.add(BatchNormalization())
sl_model.add(GRU(32))
sl_model.add(Dropout(0.2))
sl_model.add(BatchNormalization())
sl_model.add(Dense(16, activation='relu'))
sl_model.add(Dropout(0.2))
sl_model.add(BatchNormalization())
sl_model.add(Dense(1))
sl_model.compile(loss='mae', optimizer='adam')
sl_model.fit(X_trainsl_gru, y_train_sl, epochs=200, batch_size=128, verbose=2, use_multiprocessing=True, 
          validation_data=(X_testsl_gru, y_test_sl),
         callbacks=[es, mcsl])

Epoch 1/200
17/17 - 8s - loss: 284.0846 - val_loss: 326.5873
Epoch 2/200
17/17 - 1s - loss: 283.9823 - val_loss: 326.5122
Epoch 3/200
17/17 - 1s - loss: 283.9082 - val_loss: 326.3602
Epoch 4/200
17/17 - 1s - loss: 283.8444 - val_loss: 326.4265
Epoch 5/200
17/17 - 1s - loss: 283.7201 - val_loss: 326.7257
Epoch 6/200
17/17 - 1s - loss: 283.5816 - val_loss: 326.6800
Epoch 7/200
17/17 - 1s - loss: 283.4949 - val_loss: 326.6987
Epoch 8/200
17/17 - 1s - loss: 283.3654 - val_loss: 326.5738
Epoch 9/200
17/17 - 1s - loss: 283.2101 - val_loss: 326.2130
Epoch 10/200
17/17 - 1s - loss: 283.0598 - val_loss: 325.9526
Epoch 11/200
17/17 - 1s - loss: 282.8660 - val_loss: 325.9821
Epoch 12/200
17/17 - 1s - loss: 282.6482 - val_loss: 325.9539
Epoch 13/200
17/17 - 1s - loss: 282.4154 - val_loss: 325.7554
Epoch 14/200
17/17 - 1s - loss: 282.2167 - val_loss: 325.2423
Epoch 15/200
17/17 - 1s - loss: 281.9593 - val_loss: 324.6219
Epoch 16/200
17/17 - 1s - loss: 281.6870 - val_loss: 324.3797
Epoch 17/200
17/1

Epoch 133/200
17/17 - 1s - loss: 237.9181 - val_loss: 292.5412
Epoch 134/200
17/17 - 1s - loss: 237.2702 - val_loss: 292.2721
Epoch 135/200
17/17 - 1s - loss: 239.3143 - val_loss: 289.8653
Epoch 136/200
17/17 - 1s - loss: 238.0971 - val_loss: 293.8074
Epoch 137/200
17/17 - 1s - loss: 237.2554 - val_loss: 291.6493
Epoch 138/200
17/17 - 1s - loss: 234.9737 - val_loss: 289.3879
Epoch 139/200
17/17 - 1s - loss: 234.3279 - val_loss: 293.6606
Epoch 140/200
17/17 - 1s - loss: 235.0464 - val_loss: 290.0910
Epoch 141/200
17/17 - 1s - loss: 234.2966 - val_loss: 292.4181
Epoch 142/200
17/17 - 1s - loss: 233.3663 - val_loss: 289.0634
Epoch 143/200
17/17 - 1s - loss: 232.8388 - val_loss: 290.3386
Epoch 144/200
17/17 - 1s - loss: 233.5675 - val_loss: 290.6149
Epoch 145/200
17/17 - 1s - loss: 230.5228 - val_loss: 286.1985
Epoch 146/200
17/17 - 1s - loss: 231.7977 - val_loss: 290.8873
Epoch 147/200
17/17 - 1s - loss: 230.9214 - val_loss: 289.5058
Epoch 148/200
17/17 - 1s - loss: 232.3825 - val_loss: 2

In [110]:
df = pd.read_csv('last_10_test.csv')
df.Close = list(reversed(df.Close.tolist()))
df.Open = list(reversed(df.Open.tolist()))
df.High = list(reversed(df.High.tolist()))
df.Low = list(reversed(df.Low.tolist()))
df = label_calc(df)
df = indicator_fill(df)
print(len(df))
df = df[20:]
print(len(df))
df = df[df.buy_label!='Pass']

df['cl'] = df.Close
df['hi'] = df.High
df['lo'] = df.Low
df['op'] = df.Open
df['sma'] = df['20_SMA']
df['fema'] = df['5_EMA']


df['ma_diff'] = df['5_EMA'] - df['20_SMA']
df['tclose_diff'] = df['Close'] - df['20_SMA']
df['fclose_diff'] = df['Close'] - df['5_EMA']
df.Close = list(np.squeeze(cf.fit_transform(np.array(df.Close).reshape(-1,1)), axis = 1))
df.High = list(np.squeeze(cf.transform(np.array(df.High).reshape(-1,1)), axis = 1))
df.Open = list(np.squeeze(cf.transform(np.array(df.Open).reshape(-1,1)), axis = 1))
df.Low = list(np.squeeze(cf.transform(np.array(df.Low).reshape(-1,1)), axis = 1))
df['20_SMA'] = list(np.squeeze(cf.transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
df['5_EMA'] = list(np.squeeze(cf.transform(np.array(df['5_EMA']).reshape(-1,1)), axis = 1))

#df['20_SMA'] = list(np.squeeze(cf.transform(np.array(df['20_SMA']).reshape(-1,1)), axis = 1))
df = df[1:]
print(len(df))

180
160
149


In [111]:

df['us'] = df.apply(lambda x: upper_shadow(x.Open, x.Close, x.High), axis = 1)
df['ls'] = df.apply(lambda x: lower_shadow(x.Open, x.Close, x.Low), axis = 1)
df['rb'] = df.apply(lambda x: real_body(x.Open, x.Close), axis = 1)
v,bs,ss,bq,sq,tp,sl = df_vectorize(df)
df = df[10:]
v = np.array(v)




149
Vector_Done


In [112]:
v.shape

(139, 190)

In [113]:

v = np.nan_to_num(v)
v_v = v[:,40:50]
v_s = v[:,40:50]
v_bq = v[:,40:50]
v_sq = v[:,40:50]
vtp = v[:,40:50]
vsl = v[:,40:50]
bside_lab = buy_le.transform(bs)
sside_lab = sell_le.transform(ss)
bqside_lab = bq_le.transform(bq)
sqside_lab = sq_le.transform(sq)

In [114]:
print(statistics.median(tp))
print(statistics.median(sl))
print(max(tp))
print(max(sl))

61.83999999999969
54.17000000000007
217.26000000000022
202.0899999999997


In [115]:
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from tensorflow.keras.models import load_model
bmodel = load_model('best_bi-GRU_SPX_buy_model_with_stats_RealVal.h5')
smodel = load_model('best_bi-GRU_SPX_sell_model_with_stats_RealVal.h5')
bqmodel = load_model('best_bi-GRU_SPX_buy-quality_model_with_stats_RealVal.h5')
sqmodel = load_model('best_bi-GRU_SPX_sell-quality_model_with_stats_RealVal.h5')
tpmodel = load_model('best_bi-GRU_SPX_Price-Up-Move_model_with_stats_RealVal.h5')
slmodel = load_model('best_bi-GRU_SPX_Price-Down-Move_model_with_stats_RealVal.h5')
v_v = v_v.reshape(-1,v_v.shape[1],1)
v_s = v_s.reshape(-1,v_s.shape[1],1)
v_bq = v_bq.reshape(-1,v_bq.shape[1],1)
v_sq = v_sq.reshape(-1,v_sq.shape[1],1)
vtp = vtp.reshape(-1,vtp.shape[1],1)
vsl = vsl.reshape(-1,vsl.shape[1],1)

preds_out = bmodel.predict(v_v)
preds = np.argmax(preds_out, axis=1)
probs = np.max(preds_out, axis=1)

preds_out = smodel.predict(v_s)
spreds = np.argmax(preds_out, axis=1)
sprobs = np.max(preds_out, axis=1)

preds_out = bqmodel.predict(v_bq)
bqpreds = np.argmax(preds_out, axis=1)
bqprobs = np.max(preds_out, axis=1)

preds_out = sqmodel.predict(v_sq)
sqpreds = np.argmax(preds_out, axis=1)
sqprobs = np.max(preds_out, axis=1)

tppreds_out = tpmodel.predict(vtp)


slpreds_out = slmodel.predict(vsl)



print('Buy Side--------------------------')
print(accuracy_score(bside_lab, preds))
print(classification_report(bside_lab, preds))

print('Sell Side--------------------------')
print(accuracy_score(sside_lab, spreds))
print(classification_report(sside_lab, spreds))

print('Buy Quality Side--------------------------')
print(accuracy_score(bqside_lab, bqpreds))
print(classification_report(bqside_lab, bqpreds))

print('Sell Quality Side--------------------------')
print(accuracy_score(sqside_lab, sqpreds))
print(classification_report(sqside_lab, sqpreds))

Buy Side--------------------------
0.5971223021582733
              precision    recall  f1-score   support

           0       0.68      0.83      0.75        99
           1       0.06      0.03      0.03        40

    accuracy                           0.60       139
   macro avg       0.37      0.43      0.39       139
weighted avg       0.50      0.60      0.54       139

Sell Side--------------------------
0.5899280575539568
              precision    recall  f1-score   support

           0       0.29      0.30      0.30        40
           1       0.71      0.71      0.71        99

    accuracy                           0.59       139
   macro avg       0.50      0.50      0.50       139
weighted avg       0.59      0.59      0.59       139

Buy Quality Side--------------------------
0.4892086330935252
              precision    recall  f1-score   support

           0       0.54      0.56      0.55        78
           1       0.41      0.39      0.40        61

    accurac

In [116]:
df['buy_side'] = buy_le.inverse_transform(preds)
df['buy_side_orig'] = buy_le.inverse_transform(bside_lab)
df['Buy_Probs'] = probs
df['sell_side'] = sell_le.inverse_transform(spreds)
df['sell_side_orig'] = sell_le.inverse_transform(sside_lab)
df['Sell_Probs'] = sprobs
df['buy_quality'] = bq_le.inverse_transform(bqpreds)
df['bq_Probs'] = bqprobs
df['sell_quality'] = sq_le.inverse_transform(sqpreds)
df['sq_Probs'] = sqprobs
df['Predicted_upmove'] = tppreds_out
df['Predicted_downmove'] = slpreds_out
df['Actual_upmove'] = tp
df['Actual_downmove'] = sl

df.to_csv('last_10_test_output_v2.csv', index=False)

In [31]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier, MLPRegressor
xgbc = XGBClassifier(max_depth=100, objective='multi:softprob', use_label_encoder=False,num_class=2,n_estimators=2000)


xgbc.fit(X_trainb_gru.reshape(-1,X_trainb_gru.shape[1]), y_trainb,
         eval_set=[(X_testb_gru.reshape(-1,X_testb_gru.shape[1]),y_testb)]
         ,early_stopping_rounds=30)

[16:54:15] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-merror:0.47953
Will train until validation_0-merror hasn't improved in 30 rounds.
[1]	validation_0-merror:0.49123
[2]	validation_0-merror:0.45614
[3]	validation_0-merror:0.46784
[4]	validation_0-merror:0.48538
[5]	validation_0-merror:0.47368
[6]	validation_0-merror:0.47368
[7]	validation_0-merror:0.44444
[8]	validation_0-merror:0.48538
[9]	validation_0-merror:0.46784
[10]	validation_0-merror:0.46199
[11]	validation_0-merror:0.48538
[12]	validation_0-merror:0.46199
[13]	validation_0-merror:0.43860
[14]	validation_0-merror:0.46784
[15]	validation_0-merror:0.47368
[16]	validation_0-me

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=100,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=2000, n_jobs=0, num_class=2, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [22]:
import numpy as np
import h5py